In [ ]:
import pandas as pd

#데이터셋 로드
df = pd.read_csv('LOL_match data_Combined.csv')

#티어-matchID당 데이터가 10개가 아닌 경우는 제외(챔피언 정보가 다 들어가야 10개이므로)
#그냥 matchID로 보면 이상치 존재(티어는 다른데 매치ID가 같은 경우 → 데이터 수집 과정에서 발생)
df_grouped = df.groupby(['Tier','matchId']).count()
invalid_matchIds = df_grouped[df_grouped['gameType'] != 10].index.tolist()

#티어-matchID 조합에서는 이상치가 없는 것 확인
len(invalid_matchIds)

#팀 포지션이 비어있는 경우의 매치ID를 제외
matchid_invalid_position = df[df['teamPosition'].isnull()]['matchId'].unique()
df = df[~df['matchId'].isin(matchid_invalid_position)]

#경기시간 600초 초과 & 경험치 0 초과, 시즌14 마스크 적용
mask = (df['timePlayed'] > 600) & (df['champExperience'] > 0)
df_filtered = df[mask]
df_filtered['season'] = df_filtered['gameType'].str[:2]
season_mask = (df_filtered['season'] == '14')
df_filtered = df_filtered[season_mask]

# matchID별 win 컬럼이 TRUE와 FALSE의 비율이 5:5가 되는 matchId만 추출
balanced_df = df_filtered.groupby('matchId').filter(lambda x: (x['win'].sum() == 5) & (x['win'].count() - x['win'].sum() == 5))

In [ ]:

# teamPosition이 JUNGLE 또는 MIDDLE인 데이터만 추출하고 기본적인 컬럼만 포함
df_jungle_mid = balanced_df[balanced_df['teamPosition'].isin(['JUNGLE', 'MIDDLE'])][['matchId', 'teamPosition', 'championName', 'win', 'teamId']]
df_jungle_mid

In [ ]:
# df_jungle_mid의 각 matchId에 대해 조건을 확인하는 함수
def validate_match_groups(df):
    valid_match_ids = []
    for match_id, group in df.groupby('matchId'):
        if len(group) == 4:
            team_ids = group['teamId'].values
            wins = group['win'].values
            if (list(team_ids).count(100) == 2 and list(team_ids).count(200) == 2 and
                ((wins[:2] == [True, True]).all() and (wins[2:] == [False, False]).all()) or
                ((wins[:2] == [False, False]).all() and (wins[2:] == [True, True]).all())):
                valid_match_ids.append(match_id)
    return valid_match_ids

# 유효한 matchId 리스트를 확인
valid_match_ids = validate_match_groups(df_jungle_mid)

# 유효한 matchId를 가진 데이터프레임 필터링
valid_df = df_jungle_mid[df_jungle_mid['matchId'].isin(valid_match_ids)]
valid_df

In [ ]:
# 같은 matchId에서 JUNGLE과 MIDDLE의 championName 조합을 추출
combinations = []

for match_id, group in valid_df.groupby('matchId'):
    for team_id in [100, 200]:
        team_group = group[group['teamId'] == team_id]
        junglers = team_group[team_group['teamPosition'] == 'JUNGLE']['championName'].values
        middlers = team_group[team_group['teamPosition'] == 'MIDDLE']['championName'].values
        win = team_group['win'].values[0]  # 해당 팀의 승리 여부
        if len(junglers) > 0 and len(middlers) > 0:
            combinations.extend([(jungle, middle, win) for jungle in junglers for middle in middlers])

# DataFrame으로 변환
combinations_df = pd.DataFrame(combinations, columns=['JungleChampion', 'MiddleChampion', 'win'])

# 조합 별 승률 계산
win_rates = combinations_df.groupby(['JungleChampion', 'MiddleChampion']).agg(
    games_played=('win', 'count'),
    wins=('win', 'sum')
).reset_index()

win_rates['win_rate'] = (win_rates['wins'] / win_rates['games_played'] * 100 ).round(2)
win_rates = win_rates.sort_values(by='games_played', ascending=False).reset_index(drop=True)
win_rates